In [254]:
import numpy as np
import scipy.linalg
import matplotlib.pyplot as plt
%matplotlib notebook

In [294]:
# read the file
fname = "log2_continuous.txt"
f = open(fname)
lines = f.readlines()
f.close()

In [295]:
class LoggedInfo:
    def __init__(self, states, foot_locs, inputs, ones, X_s, y_s):
        self.states = states
        self.foot_locs = foot_locs
        self.inputs = inputs
        self.ones = ones
        self.X_s = X_s
        self.y_s = y_s
        
# get the A, B, and b matrices
def get_matrix(lines, cur_pointer):
    tmp = []
    while cur_pointer < len(lines) and lines[cur_pointer][0] != "#":
        tmp.append(lines[cur_pointer])
        cur_pointer += 1
    
    matrix = []
    for line in tmp:
        l_split = line.strip("\n").split(" ")
        matrix.append([float(s) for s in l_split if s != ""])
    return matrix, cur_pointer+1

## Assumes that you start from the first line of 
def get_logged_states_and_matrices(lines, cur_pointer):
    p = cur_pointer
    states = []
    foot_locs = []
    inputs = []
    ones = []  # this is just for convenience
    X_s = []
    y_s = []
    while p < len(lines):
        # get state shit
        vec_row, p = get_matrix(lines, p)
        vec_row = (np.array(vec_row).T)[0]
        states.append(vec_row[0:12])
        foot_locs.append(vec_row[12:15])
        inputs.append(vec_row[15:19])
        ones.append(vec_row[19:])
        
        # get X_
        X_, p = get_matrix(lines, p)
        X_s.append(np.array(X_))

        # get y_
        y_, p = get_matrix(lines, p)
        y_s.append(np.array(y_))

        if p + 100 + 100 + 31 >= len(lines):
            break
    return LoggedInfo(states, foot_locs, inputs, ones, X_s, y_s)

In [296]:
A, cp = get_matrix(lines, 1)
B, cp = get_matrix(lines, cp)
b, cp = get_matrix(lines, cp)
logged_stuff = get_logged_states_and_matrices(lines, cp)

In [297]:
print(B)

[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0333091, 0.0, 0.0, 0.0], [0.0, 0.0333091, 0.0, 0.0], [0.0, 0.0, 0.0333091, 0.0], [0.0, 0.76044, 0.0, 0.0], [-1.25818, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.1236]]


In [298]:
print(A)

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, -323.642, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 323.642, 0.0], [535.48, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -535.48, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]


In [299]:
states = logged_stuff.states
inputs = logged_stuff.inputs
foot_locs = logged_stuff.foot_locs
ones = logged_stuff.ones
X_s = logged_stuff.X_s
y_s = logged_stuff.y_s
A = np.array(A)
B = np.array(B)
b = np.array(b).squeeze()

In [300]:
idx = 1
print(states[idx])
print(X_s[idx][49][:12])
print(y_s[idx][49])

joined = np.concatenate([states[idx], foot_locs[idx]])
print(A.dot(joined) + B.dot(inputs[idx]) + b)

[ 9.50469e-07  2.71230e-12  6.92000e-01  2.68644e-08  3.16247e-06
 -2.48071e-08  2.85054e-03  3.10610e-06  5.48277e-04  1.22734e-04
  1.24677e-02 -1.23152e-04]
[ 9.50469e-07  2.71230e-12  6.92000e-01  2.68644e-08  3.16247e-06
 -2.48071e-08  2.85054e-03  3.10610e-06  5.48277e-04  1.22734e-04
  1.24677e-02 -1.23152e-04]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[ 2.85054000e-03  3.10610000e-06  5.48277000e-04  1.22734000e-04
  1.24677000e-02 -1.23152000e-04  4.11833712e-01 -1.62176678e+00
 -3.72260000e-06 -2.08425716e+01 -1.05306093e+01  3.68641924e-25]


In [15]:
solve_idx = 105
X_c = X_s[solve_idx][:99, :-11]
y_c = y_s[solve_idx][:99]

# soln = scipy.linalg.solve(X_c.T.dot(X_c) + np.eye(X_c.shape[1]) * 0.1, X_c.T.dot(y_c)).T
soln = (np.linalg.pinv(X_c.T @ X_c) @ X_c.T @ y_c).T
A_hat = soln[:,:15]
B_hat = soln[:,15:15+4]
b_hat = soln[:,-1]
print(A_hat)

[[ 0.00000000e+00  0.00000000e+00 -7.69917918e-03 -2.20195541e-06
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -5.66791776e-03 -1.15426789e-06
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  6.45511228e-04 -2.79315174e-07
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -1.03224063e-01 -2.06774479e-05
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -3.32268400e-01 -7.54134840e-05
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -2.25814784e-02 -4.94933866e-06
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -1.93699597e+01 -6.22328307e-03
   0.00000000e+00  0.0000000

In [202]:
idx = 5
# print(states[idx])
# print(X_s[idx][99][:12])
# print(y_s[idx][99])

joined = np.concatenate([states[idx], foot_locs[idx]])
nom_and_res = A.dot(joined) + B.dot(inputs[idx]) + b + A_hat.dot(joined) + B_hat.dot(inputs[idx]) + b_hat
print(nom_and_res[:5])
print(states[idx+1][:5])
print(states[idx][:5])

[3.58083719e-05 1.68486031e-05 7.39950381e-06 2.05279839e-05
 2.26547264e-05]
[ 2.82424e-05 -4.04471e-06  5.71984e-01 -7.39821e-05 -2.67601e-04]
[ 2.61984e-05 -2.29849e-06  5.71986e-01 -4.16516e-05 -1.57322e-04]


In [200]:
print(A[:5,:5])
print(A_hat[:5,:5])

[[ 1.         0.         0.         0.         0.       ]
 [ 0.         1.         0.         0.         0.       ]
 [ 0.         0.         1.         0.         0.       ]
 [ 0.        -0.0161821  0.         1.         0.       ]
 [ 0.026774   0.         0.         0.         1.       ]]
[[ 1.10136794e-03  3.03234726e-04  1.39777190e-02 -9.51748118e-02
  -5.24095951e-03]
 [ 5.20310790e-03  2.13305996e-04  1.16705888e-02 -9.04072310e-02
   3.32390173e-02]
 [ 6.73555137e-03  1.04249271e-04 -2.48355401e-01 -6.90001890e-03
   7.60215637e-03]
 [-4.44675048e-03 -4.30475060e-03  9.08404415e-02 -1.07683545e+00
   4.16315231e-02]
 [-4.48001798e-02 -1.09355800e-02 -2.44867981e-02 -8.91673973e-02
  -9.52022939e-01]]


In [189]:
print(np.linalg.cond(X_c.T.dot(X_c)))

2.9476103716545757e+21


In [201]:
print(inputs[0:10])

[array([0., 0., 0., 0.]), array([ 1.23640e+01, -4.86884e+01,  3.01982e+02, -2.10862e-32]), array([ 1.23640e+01, -4.86884e+01,  3.01982e+02, -2.10862e-32]), array([ 1.23640e+01, -4.86884e+01,  3.01982e+02, -2.10862e-32]), array([ 1.23640e+01, -4.86884e+01,  3.01982e+02, -2.10862e-32]), array([ 1.23640e+01, -4.86884e+01,  3.01982e+02, -2.10862e-32]), array([ 1.23640e+01, -4.86884e+01,  3.01982e+02, -2.10862e-32]), array([ 1.23640e+01, -4.86884e+01,  3.01982e+02, -2.10862e-32]), array([ 1.23640e+01, -4.86884e+01,  3.01982e+02, -2.10862e-32]), array([ 1.23640e+01, -4.86884e+01,  3.01982e+02, -2.10862e-32])]


In [7]:
# check that y is constructed properly
idx = 1
y_ = y_s[idx]
state = states[idx]
foot_loc = foot_locs[idx]
input = inputs[idx]

# check that y_ is consistent from step to step.
# print(y_[-1] + states[idx + 1])
# print(A @ np.concatenate([state, foot_loc]) + B @ input + b - states[idx+1])
# print(y_s[idx+1][-2])

print(y_s[idx+1][-2] - y_s[idx][-1])
print(states[idx])
print(states[idx+1])

[ 7.00914e-04  1.19183e-06 -3.75794e-04  4.83094e-05  1.93984e-03
 -3.71709e-05  2.57574e+00  1.62610e+00 -1.54147e+00  1.13164e+01
  1.75273e+01 -1.37955e-01]
[ 1.44476e-06  1.77440e-11  5.72000e-01  2.67611e-08  3.34059e-06
 -1.50560e-08  3.51666e-03  1.48848e-06  4.03165e-04  9.52683e-05
  1.08480e-02 -6.49444e-05]
[ 3.55355e-06  1.35790e-09  5.72000e-01  9.85499e-08  9.73449e-06
 -6.61137e-08  5.01045e-03  3.65619e-06 -2.43187e-04  1.86832e-04
  1.41826e-02 -1.33922e-04]


In [227]:
## TRY this from scratch now with this data we have here
size = 50
X_new = np.zeros((size, 12 + 3 + 4 + 1))
y_new = np.zeros((size, 12))

for i in range(0, size):
    state = states[idx]
    foot_loc = foot_locs[idx]
    input = inputs[idx]
    
    X_new[0:size-1] = X_new[1:size]
    y_new[0:size-1] = y_new[1:size]
    next_state_nom = A @ np.concatenate([state, foot_loc]) + B @ input + b
    y_new[size-1] = -next_state_nom
    X_new[size-1] = np.concatenate([state, foot_loc, input, np.ones(1)])
    y_new[size-2] += state

In [228]:
X_c = X_new[:-1]
y_c = y_new[:-1]
soln = (np.linalg.pinv(X_c.T @ X_c) @ X_c.T @ y_c).T
A_hat = soln[:,:15]
B_hat = soln[:,15:15+4]
b_hat = soln[:,-1]

idx = 5
# print(states[idx])
# print(X_s[idx][99][:12])
# print(y_s[idx][99])

joined = np.concatenate([states[idx], foot_locs[idx]])
nom_and_res = A.dot(joined) + B.dot(inputs[idx]) + b + A_hat.dot(joined) + B_hat.dot(inputs[idx]) + b_hat
print(np.linalg.norm(nom_and_res - states[idx + 1]))
print(np.linalg.norm(states[idx] - states[idx + 1]))

0.06466666217992227
0.06466666217992233


In [8]:
## doing it in the continuous time mode

In [301]:
size = 50
X_new = np.zeros((size, 12 + 3 + 4 + 1))
y_new = np.zeros((size, 12))
prev_state = np.zeros(12 + 3)
prev_input = np.zeros(4)

for i in range(0, size):
    state = states[i]
    foot_loc = foot_locs[i]
    input = inputs[i]
    
    X_new[0:size-1] = X_new[1:size]
    y_new[0:size-1] = y_new[1:size]
    next_state_nom = A @ prev_state + B @ prev_input + b
    X_new[size-1] = np.concatenate([state, foot_loc, input, np.ones(1)])
    y_new[size-2] = (state - prev_state[:12]) * 2000 - next_state_nom
    if i < 10:
        print(f"ydot at iteration {i} = {(state - prev_state[:12]) * 2000}")
    if i == 40:
        # print(f"at iteration {i} prev_state = {prev_state}")
        # print(f"at iteration {i} nominal_DERIV = {next_state_nom}")
        print(f"at iteration {i} y_new(size - 2) = {y_new[size-2]}")
        print(f"at iteration {i}, y_s[i][size - 2] = {y_s[i][size-2]}")
    prev_state = np.concatenate([state, foot_loc])
    prev_input = input
    
X_c = X_new[:-1]
y_c = y_new[:-1]
soln = (np.linalg.pinv(X_c.T @ X_c) @ X_c.T @ y_c).T
A_hat_p = soln[:,:15]
B_hat_p = soln[:,15:15+4]
b_hat_p = soln[:,-1]

ydot at iteration 0 = [ 0.00000e+00  0.00000e+00  1.38400e+03 -8.99910e-10  3.35444e-10
 -7.54782e-23  0.00000e+00  0.00000e+00  0.00000e+00  0.00000e+00
  0.00000e+00  0.00000e+00]
ydot at iteration 1 = [ 1.90093800e-03  5.42460000e-09  0.00000000e+00  5.37296999e-05
  6.32493966e-03 -4.96142000e-05  5.70108000e+00  6.21220000e-03
  1.09655400e+00  2.45468000e-01  2.49354000e+01 -2.46304000e-01]
ydot at iteration 2 = [ 2.16980620e-02  3.90956575e-04 -1.48000000e-01  1.17769512e-02
 -1.15714894e+00 -1.10769058e-02 -2.64612800e+01  7.03450000e-02
 -5.68333540e+01  1.70355400e+00 -5.70821400e+02 -1.49642200e+00]
ydot at iteration 3 = [-1.1413660e-02 -6.0607800e-04 -2.8000000e-02 -1.5755920e-02
 -3.0954400e-01 -4.7642200e-03 -4.1300000e+00 -2.5406972e+00
 -1.9272000e+00 -6.7192222e+01 -1.4492000e+02 -1.5509794e+01]
ydot at iteration 4 = [-1.3589018e-02 -1.8841840e-03 -3.0000000e-02 -5.0248560e-02
 -3.8302600e-01 -1.2515260e-02 -4.5344000e+00 -2.5599600e+00
 -2.2754000e+00 -6.9108400e+01 -

In [304]:
idx = 60
print("actual state derivative:")
print((states[idx+1] - states[idx])*2000)
print("estimated derivative")
print((A + A_hat_p) @ np.concatenate([states[idx], foot_locs[idx]]) + (B + B_hat_p) @ inputs[idx] + (b + b_hat_p))
print("nominal derivative:")
print(A @ np.concatenate([states[idx], foot_locs[idx]]) + B @ inputs[idx] + b)

actual state derivative:
[-7.8380e-02 -2.8880e-02 -1.2400e-01 -7.0680e-01 -2.7060e+00 -1.5600e-02
 -9.5280e-01 -7.4880e-01 -9.9400e-01 -1.1298e+01  8.4800e+00  1.9877e+01]
estimated derivative
[ 2.22429120e+00  8.91001025e-01  7.03750444e+00  2.38892828e+01
  1.05813316e+02  2.12954885e+01  1.61400130e+02 -6.71468645e+00
  9.28868080e+01 -5.11519467e+02  2.14755327e+03 -6.85353108e+02]
nominal derivative:
[-7.81278000e-02 -2.87028000e-02 -1.23949000e-01 -7.38505000e-01
 -2.69729000e+00 -1.01647000e-01 -2.08149898e+00 -1.03956369e+00
  2.00850150e-01 -7.12460756e+00  8.20258349e+01  1.01708946e+00]


In [142]:
idx = 10
state = states[idx]
state_next = states[idx + 1]
input = inputs[idx]
foot_loc = foot_locs[idx]
next_state_nom = A @ np.concatenate([state, foot_loc]) + B @ input + b

print(y_s[99][idx])
print(y_new[idx])
print((state_next - state) * 2000 - next_state_nom)

[-1.38185e-03 -6.13370e-04 -6.99198e-04 -1.61122e-02 -3.97816e-02
 -3.60880e-03 -5.03184e+00 -4.02162e+00 -2.78270e+00 -1.16586e+02
 -1.76600e+02 -1.60885e+01]
[-1.38182000e-03 -6.13530000e-04  1.06000000e-04 -1.61110000e-02
 -3.97790000e-02 -3.60830000e-03 -5.85543371e+00 -7.78093216e-01
 -2.78259628e+00 -4.25363092e+01 -1.45487122e+02 -1.60884000e+01]
[-1.38182000e-03 -6.13530000e-04  1.06000000e-04 -1.61110000e-02
 -3.97790000e-02 -3.60830000e-03 -5.85543371e+00 -7.78093216e-01
 -2.78259628e+00 -4.25363092e+01 -1.45487122e+02 -1.60884000e+01]


In [267]:
X_c = X_s[100][:-1]
y_c = y_s[100][:-1]
soln = (np.linalg.pinv(X_c.T @ X_c) @ X_c.T @ y_c).T
A_hat = soln[:,:15]
B_hat = soln[:,15:15+4]
b_hat = soln[:,-1]

idx = 10
print("actual state derivative:")
print((states[idx+1] - states[idx])*2000)
print("estimated derivative")
print((A + A_hat) @ np.concatenate([states[idx], foot_locs[idx]]) + (B + B_hat) @ inputs[idx] + (b + b_hat))
print("nominal derivative:")
print(A @ np.concatenate([states[idx], foot_locs[idx]]) + B @ inputs[idx] + b)

actual state derivative:
[-1.269692e-02 -1.056680e-02 -1.200000e-02 -2.829880e-01 -6.666000e-01
 -6.655600e-02 -5.337000e+00 -2.398000e+00 -2.505200e+00 -6.333800e+01
 -1.555920e+02 -1.613180e+01]
estimated derivative
[ 6.53753447e-03 -2.10840401e-03  6.41298525e-03 -6.99319020e-02
 -2.24009998e-02 -6.66973937e-02 -5.72452956e+00 -2.49598480e+00
 -9.11857631e-01 -6.13867048e+01 -1.50297088e+02 -1.76053587e+01]
nominal derivative:
[-1.13440000e-02 -9.95390000e-03 -1.25879000e-02 -2.66879000e-01
 -6.26949000e-01 -6.28880000e-02  4.11833712e-01 -1.62176678e+00
 -3.72260000e-06 -2.08356921e+01 -1.05251123e+01  3.68641924e-25]


In [121]:
# testing that the y_s and X_s matrices are built correctly
idx = 12
state = states[idx]
state_next = states[idx + 1]
input = inputs[idx]
foot_loc = foot_locs[idx]
next_state_nom = A @ np.concatenate([state, foot_loc]) + B @ input + b

print(y_s[idx+1][-2])
# print(state_next)
# print(state)
# print(next_state_nom)
print((state_next - state) * 2000 - next_state_nom)

[-8.59393e-04 -4.89420e-04 -1.38407e-03 -1.09150e-02 -2.26875e-02
  1.80164e-03 -3.43485e+00 -3.40890e+00 -5.35370e+00 -9.64697e+01
 -8.68862e+01 -1.25128e+01]
[-8.58900000e-04 -4.90300000e-04 -1.97640000e-03 -1.09290000e-02
 -2.26900000e-02  1.80200000e-03 -3.17643164e+00 -3.97018978e-01
 -5.46388953e+00 -2.77084403e+01 -9.66347468e+01 -1.28554063e+01]


In [164]:
print(y_new[-2])

[ 6.83900000e-04 -2.10700000e-04 -1.25800000e-03  1.42790000e-02
  1.48300000e-02  1.49860400e-01  5.14705824e+00  9.75770333e-03
 -2.52765248e+00  5.35806460e+00  1.02623014e+01  1.01131160e+01]


In [169]:
print(y_s[99][-2])

[ 6.83057e-04 -2.02820e-04 -5.98790e-04  1.43622e-02  1.62250e-02
  1.49892e-01  2.48877e-01 -1.60003e+00 -2.13414e+00 -3.13964e+01
  1.95284e+02  1.01917e+01]


In [269]:
prev_state = np.array([-0.000423056,
-0.000257203,
    0.691601,
 -0.00664396,
  -0.0170387,
 -0.00186588,
  -0.0602159,
  -0.0294357,
  -0.0544914,
   -0.744674,
    -2.00513,
   -0.251766,
 -0.00938596,
   0.0500047,
-8.74176e-06])


prev_input = np.array([-13.1455,
-37.2995,
 295.339,
 0.46863])

In [272]:
# print(np.dot(A, prev_state))
# print(np.dot(B, prev_input))
# print(b)
print(np.dot(A, prev_state) + np.dot(B, prev_input) + b)

[ -0.0602159   -0.0294357   -0.0544914   -0.744674    -2.00513
  -0.251766    -0.43786477  -1.24241278   0.02747628 -12.09716897
  21.33886102   0.52655267]


In [246]:
print(b)

[ 0.    0.    0.    0.    0.    0.   -0.   -0.   -9.81  0.    0.    0.  ]


In [ ]:
nominal deriv: 
-0.0602159
-0.0294357
-0.0544914
 -0.744674
  -2.00513
 -0.251766
 -0.437866
  -1.24241
 0.0274927
  -12.0972
   21.3389
  0.526551